In [1]:
import cv2
import numpy as np

In [13]:
# https://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/

image = cv2.imread("F:\Colonies_Data\data\plate 2_24.png", )
image = cv2.resize(image, (300,300))
output = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, minDist=150, dp=2, minRadius=150, maxRadius=250 )
 
# ensure at least some circles were found
if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    circle_ = np.round(circles[0, :]).astype("int")
 
    # loop over the (x, y) coordinates and radius of the circles
    for (x, y, r) in circle_:
        # draw the circle in the output image, then draw a rectangle
        # corresponding to the center of the circle
        cv2.circle(output, (x, y), r, (0, 255, 0), 4)
        cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
 
    # show the output image
    cv2.imshow("output", np.hstack([image, output]))
    cv2.waitKey(0)

In [14]:
circles

array([[[149. , 149. , 150.3]]], dtype=float32)